# Chapter 6: Lowering to RISC-V

In [1]:
from compiler import (parse_toy, print_op, optimise_toy, lower_from_toy, 
                                  optimise_vir, lower_to_riscv, print_riscv_ssa, emulate_riscv)

example = """
def main() {
  var a<2, 3> = [[1, 2, 3], [4, 5, 6]];
  var b<6> = [1, 2, 3, 4, 5, 6];
  var c<2, 3> = b;
  var d = a + c;
  print(d);
}
"""

toy_0 = parse_toy(example)
toy_1 = optimise_toy(toy_0)
vir_0 = lower_from_toy(toy_1)
vir_1 = optimise_vir(vir_0)
riscv = lower_to_riscv(vir_1)
code = print_riscv_ssa(riscv)

print(code)

.bss 
heap:
.space 1024
.data 
main.tensor_shape.0:
.word 0x2, 0x2, 0x3
main.tensor_data.0:
.word 0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6
main.tensor_shape.1:
.word 0x2, 0x2, 0x3
main.tensor_data.1:
.word 0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6
.text 
main:
	li	%0, heap
	li	%1, main.tensor_shape.0
	li	%2, main.tensor_data.0
	li	%3, main.tensor_shape.1
	li	%4, main.tensor_data.1
	lw	%5, %2, 0		# Get input count
	addi	%6, %5, 1		# Input storage int32 count
	buffer.alloc	%7, %6
	sw	%5, %7, 0		# Set result count
	addi	%8, %2, 4		# lhs storage
	addi	%9, %4, 4		# lhs storage
	addi	%10, %7, 4		# destination storage
	buffer.add	%5, %8, %10
	buffer.add	%5, %9, %10
	li	%11, 2
	buffer.alloc	%12, %11
	sw	%1, %12, 0		# Set tensor shape
	lw	%13, %12, 0
	sw	%7, %12, 4		# Set tensor data
	toy.print	%12
	li	a7, 93
	scall



In [2]:
emulate_riscv(code)

[CPU] Started running from example.asm:.text at heap (0x100) + 0x450
Program(name=example.asm,sections=set(),base=['.bss', '.data', '.text'])
[[2, 4, 6], [8, 10, 12]]
[CPU] Program exited with code 0
